In [1]:
import h5py

from underworld import UWGeodynamics as GEO
from underworld import visualisation as vis

import underworld as uw
from underworld import function as fn

import numpy as np
import math

import matplotlib.pyplot as plt
import imageio 

loaded rc file /opt/venv/lib/python3.10/site-packages/underworld/UWGeodynamics/uwgeo-data/uwgeodynamicsrc


In [2]:
#!pip install imageio

In [3]:
u = GEO.UnitRegistry

KL = 500 * u.kilometer
K_viscosity = 1e20  * u.pascal * u.second
K_density   = 3200 * u.kilogram / u.meter**3

KM = K_density * KL**3
Kt = KM/ ( KL * K_viscosity )

GEO.scaling_coefficients["[length]"] = KL
GEO.scaling_coefficients["[time]"] = Kt
GEO.scaling_coefficients["[mass]"]= KM

In [4]:
nx0 = 50
ny0 = 50

Model = GEO.Model(elementRes=(nx0,ny0),
                  minCoord=(-250. * u.kilometer, -500. * u.kilometer),  
                  maxCoord=(250. * u.kilometer, 0. * u.kilometer),
                  gravity=(0.0, -9.81 * u.meter / u.second**2))

# dt = 2.5*u.kiloyear
# dt_str = "%.1f" %(dt.m)
# checkpoint_interval = 1e2*u.kiloyear
# fdir = "1_23_02_FreeSurface_Kaus2010_Rayleigh-Taylor_Instability_dt"+dt_str+"ka_NUM"
# Model.outputDir = fdir

	Global element size: 50x50
	Local offset of rank 0: 0x0
	Local range of rank 0: 50x50
In func WeightsCalculator_CalculateAll(): for swarm "SU3EFPYX__swarm"
	done 33% (834 cells)...
	done 67% (1667 cells)...
	done 100% (2500 cells)...
WeightsCalculator_CalculateAll(): finished update of weights for swarm "SU3EFPYX__swarm"


In [5]:
minCoord = tuple([GEO.nd(val) for val in Model.minCoord])
maxCoord = tuple([GEO.nd(val) for val in Model.maxCoord])

mesh1 = uw.mesh.FeMesh_Cartesian(elementType=Model.elementType,
                                    elementRes=Model.elementRes,
                                    minCoord=minCoord,
                                    maxCoord=maxCoord,
                                    periodic=Model.periodic)

	Global element size: 50x50
	Local offset of rank 0: 0x0
	Local range of rank 0: 50x50


In [6]:
def PlotModelSetup(fdir,meshCopy,step,dt,field_name,remesh=True):
    step = int(step)

    if remesh:
        meshfile = fdir+"mesh-"+ str(step)+".h5"
        meshCopy.load(meshfile)
    else:
        meshfile = fdir+"mesh"+".h5"
        meshCopy.load(meshfile)

    swarmCopy = uw.swarm.Swarm(mesh=meshCopy,particleEscape=True)
    swarmfile = fdir+"swarm-"+ str(step)+".h5"
    swarmCopy.load(swarmfile)

    materialField = swarmCopy.add_variable("int",1)
    filename = fdir+"materialField-"+ str(step)+".h5"
    materialField.load(filename)
    
    figname = fdir + field_name +"-{0}.png".format(step)
    time_str = str(step*dt)
    Fig = vis.Figure(figsize=(500,500), quality=3)
    Fig.Mesh(meshCopy)
#     if remesh:
#         colours= ["orange","yellow","red"]
#     else:
#         colours= ["blue","orange","yellow","red"]
    
#    Fig.Points(swarmCopy,materialField,fn_size=3.0,colourBar=False,colours= colours)
    #Fig.show()
    
    Fig.Points(swarmCopy,materialField,fn_size=3.0,colourBar=False)
    Fig.save(fdir+field_name+"-{0}.png".format(step))

In [7]:
fdir1 = "1_23_02_FreeSurface_Kaus2010_Rayleigh-Taylor_Instability_dt2.5ka/"
fdir2 = "1_23_02_FreeSurface_Kaus2010_Rayleigh-Taylor_Instability_dt2.5ka_NUM/"


PlotModelSetup(fdir1,mesh1,0,dt,"ModelSetup",remesh = True)
PlotModelSetup(fdir2,mesh1,0,dt,"ModelSetup",remesh = True)

In [8]:
def PlotField(fdir,meshCopy,step,dt,field_name,remesh=True):
    step = int(step)

    if remesh:
        meshfile = fdir+"mesh-"+ str(step)+".h5"
        swarmCopy = uw.swarm.Swarm(mesh=meshCopy,particleEscape=True)
        meshCopy.load(meshfile)
    else:
        meshfile = fdir+"mesh"+".h5"
        swarmCopy = uw.swarm.Swarm(mesh=meshCopy,particleEscape=True)
        meshCopy.load(meshfile)

    swarmfile = fdir+"swarm-"+ str(step)+".h5"
    swarmCopy.load(swarmfile)

    materialField = swarmCopy.add_variable("int",1)
    filename = fdir+"materialField-"+ str(step)+".h5"
    materialField.load(filename)

    velocityField = meshCopy.add_variable(nodeDofCount=2)
    filename = fdir+"velocityField-"+ str(step)+".h5"
    velocityField.load(filename)
    
    figname = fdir + field_name +"-{0}.png".format(step)
    time_str = str(step*dt/1000)
    Fig = vis.Figure(figsize=(500,500),title=time_str+" ma", quality=3)
    Fig.Mesh(meshCopy)
#     if remesh:
#         colours= ["orange","yellow","red"]
#     else:
#         colours= ["blue","orange","yellow","red"]
    
    
#     Fig.Points(swarmCopy,materialField,fn_size=3.0,colourBar=False,colours= colours)

    Fig.Points(swarmCopy,materialField,fn_size=3.0,colourBar=False)
    Fig.VectorArrows(meshCopy, velocityField)
    #Fig.show()
    Fig.save(fdir+field_name+"-{0}.png".format(step))

In [11]:
dt = 100  #ka
steps_arange = np.arange(0,54+1,1)


fdir = fdir1
field_name = "VelocityField"

for step in steps_arange:
    PlotField(fdir,mesh1,step,dt,field_name,remesh = True)
    
images = [] 
for step in steps_arange:
    f_name = fdir+field_name+"-{0}.png".format(step)
    images.append(imageio.imread(f_name)) 
imageio.mimsave(fdir+field_name+".gif", images, duration=1.0) 

In [12]:
fdir = fdir2
field_name = "VelocityField"

for step in steps_arange:
    #print(step)
    PlotField(fdir,mesh1,step,dt,field_name,remesh = True)
    
images = [] 
for step in steps_arange:
    f_name = fdir+field_name+"-{0}.png".format(step)
    images.append(imageio.imread(f_name)) 
imageio.mimsave(fdir+field_name+".gif", images, duration=1.0)